In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("reduction-op")
sc = SparkContext(conf=conf)
sc

# Reduce
- `RDD.reduce(<task>)`
- 사용자가 지정하는 task를 받아 여러 개의 값을 하나로 줄여준다.

In [2]:
sample_rdd= sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
sample_rdd.reduce(lambda x, y : x + y)

15

## 파티션에 따라 결과물이 달라진다.
- 분산된 파티션들의 연산과 합치는 부분을 나눠서 생각해야 함

사용할 TASK
```python
lambda x, y : (x * 2) + y
```

In [4]:
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y : (x * 2) + y)

26

In [5]:
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

18

In [6]:
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : (x * 2) + y)

18

In [7]:
sc.parallelize([1, 2, 3, 4], 4).reduce(lambda x, y : (x * 2) + y)

26

# Fold
- `RDD.fold(zeroValue, <task>)`
- `reduce`와 비슷하지만, 시작값을 zeroValue에 넣어놓고 `reduce`할 수 있다.

In [8]:
rdd = sc.parallelize([2, 3, 4], 4)
rdd.glom().collect()

[[], [2], [3], [4]]

In [9]:
rdd.reduce(lambda x , y : x * y) # reduce는 없는 값은 없는것처럼 생각

24

In [10]:
rdd.fold(1, lambda x, y : x * y) # fold는 없는 값에 강제로 넣어서 연산 생각

24

In [12]:
rdd.reduce(lambda x , y : x + y)

9

In [14]:
rdd.fold(1, lambda x , y : x + y)

14

# Aggregate( Action )
- `RDD.aggregate(zeroValue, seqOp, combOp)`
    - `zeroValue` : 각 파티션에서 누적할 시작값
    - `seqOp` : 타입 변경 함수. 파티션 내부에서 사용할 task
    - `combOp` : 합치는 함수. 파티션 끼리 사용할 task

In [16]:
rdd = sc.parallelize([1, 2, 3, 4], 2)
rdd.glom().collect()

[[1, 2], [3, 4]]

In [18]:
seqOp = lambda x, y : (x[0] + y, x[1] + 1) # 파티션 내의 reduce task
combOp = lambda x , y : (x[0] + y[0], x[1] + y[1]) # 파티션 연산 결과에 대한 reduce task

In [19]:
rdd.aggregate((0, 0), seqOp, combOp)

(10, 4)

In [20]:
sc.stop()